The code in this colab notebook is heavily referenced from this [blog post by Usman Malik](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/).

# Imports and Setups

In [1]:
!pip install -qq wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np

Create a Weigts and Biases account if you don't already have one: https://wandb.ai/signup

Copy and paste the API token in the input box below: https://wandb.ai/authorize

In [3]:
import wandb
from wandb.keras import WandbMetricsLogger

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# One to Many Sequence

## Data

In [4]:
X, Y = [], []
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = []
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43]
[[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], [26, 27], [29, 30], [32, 33], [35, 36], [38, 39], [41, 42], [44, 45]]


Let's reshape the input sequence to be of the shape `[batch, timesteps, feature]`.

In [5]:
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

print(f"Shape of X: {X.shape} and shape of Y: {Y.shape}")

Shape of X: (15, 1, 1) and shape of Y: (15, 2)


## Model

In [6]:
def get_model():
    inputs = layers.Input(shape=(1,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(2)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                                 
Total params: 10502 (41.02 KB)
Trainable params: 10502 (41.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train with W&B

In [7]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

wandb: Currently logged in as: shridevi-s (vitchennai). Use `wandb login --relogin` to force relogin


Epoch 1/1000
4/4 [==============================] - 4s 96ms/step - loss: 476.8731 - val_loss: 1742.1198
Epoch 2/1000
4/4 [==============================] - 0s 12ms/step - loss: 470.6498 - val_loss: 1724.7572
Epoch 3/1000
4/4 [==============================] - 0s 11ms/step - loss: 465.9904 - val_loss: 1707.2617
Epoch 4/1000
4/4 [==============================] - 0s 11ms/step - loss: 459.6311 - val_loss: 1689.6221
Epoch 5/1000
4/4 [==============================] - 0s 11ms/step - loss: 454.2057 - val_loss: 1670.1356
Epoch 6/1000
4/4 [==============================] - 0s 11ms/step - loss: 448.9332 - val_loss: 1648.3136
Epoch 7/1000
4/4 [==============================] - 0s 11ms/step - loss: 441.7359 - val_loss: 1625.0280
Epoch 8/1000
4/4 [==============================] - 0s 13ms/step - loss: 434.3073 - val_loss: 1599.6359
Epoch 9/1000
4/4 [==============================] - 0s 11ms/step - loss: 426.3857 - val_loss: 1571.4717
Epoch 10/1000
4/4 [==============================] - 0s 10ms/ste

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,3998
batch/learning_rate,0.001
batch/loss,0.00732
epoch/epoch,999


## Prediction

In [8]:
test_input = np.array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

1/1 [==============================] - 0s 172ms/step
[[10.9993725 12.12687  ]]


# Many to One Sequence

## Data

In [9]:
X = np.array([x+1 for x in range(45)])
X = X.reshape(15,3,1)

Y = []
for x in X:
    Y.append(x.sum())
Y = np.array(Y)

print(X)
print(Y)

[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]

 [[ 7]
  [ 8]
  [ 9]]

 [[10]
  [11]
  [12]]

 [[13]
  [14]
  [15]]

 [[16]
  [17]
  [18]]

 [[19]
  [20]
  [21]]

 [[22]
  [23]
  [24]]

 [[25]
  [26]
  [27]]

 [[28]
  [29]
  [30]]

 [[31]
  [32]
  [33]]

 [[34]
  [35]
  [36]]

 [[37]
  [38]
  [39]]

 [[40]
  [41]
  [42]]

 [[43]
  [44]
  [45]]]
[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


## Model

In [10]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(1)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train

In [11]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 3745.1389 - val_loss: 14076.3545
Epoch 2/1000
1/1 [==============================] - 0s 37ms/step - loss: 3722.6477 - val_loss: 13990.2998
Epoch 3/1000
1/1 [==============================] - 0s 45ms/step - loss: 3699.0742 - val_loss: 13898.1445
Epoch 4/1000
1/1 [==============================] - 0s 34ms/step - loss: 3674.3547 - val_loss: 13799.7461
Epoch 5/1000
1/1 [==============================] - 0s 34ms/step - loss: 3648.3713 - val_loss: 13694.7578
Epoch 6/1000
1/1 [==============================] - 0s 39ms/step - loss: 3621.0039 - val_loss: 13582.5547
Epoch 7/1000
1/1 [==============================] - 0s 34ms/step - loss: 3592.1199 - val_loss: 13462.4375
Epoch 8/1000
1/1 [==============================] - 0s 35ms/step - loss: 3561.5637 - val_loss: 13333.6221
Epoch 9/1000
1/1 [==============================] - 0s 33ms/step - loss: 3529.1633 - val_loss: 13195.2236
Epoch 10/1000
1/1 [=============================

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,1998
batch/learning_rate,0.001
batch/loss,0.00929
epoch/epoch,999


## Prediction

In [12]:
test_input = np.array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[151.68916]]


# Many to Many Sequence

## Data

In [13]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

print(X)
print(Y)

[[[  5]
  [ 10]
  [ 15]]

 [[ 20]
  [ 25]
  [ 30]]

 [[ 35]
  [ 40]
  [ 45]]

 [[ 50]
  [ 55]
  [ 60]]

 [[ 65]
  [ 70]
  [ 75]]

 [[ 80]
  [ 85]
  [ 90]]

 [[ 95]
  [100]
  [105]]

 [[110]
  [115]
  [120]]

 [[125]
  [130]
  [135]]

 [[140]
  [145]
  [150]]

 [[155]
  [160]
  [165]]

 [[170]
  [175]
  [180]]

 [[185]
  [190]
  [195]]

 [[200]
  [205]
  [210]]

 [[215]
  [220]
  [225]]

 [[230]
  [235]
  [240]]

 [[245]
  [250]
  [255]]

 [[260]
  [265]
  [270]]

 [[275]
  [280]
  [285]]

 [[290]
  [295]
  [300]]]
[[[ 20]
  [ 25]
  [ 30]]

 [[ 35]
  [ 40]
  [ 45]]

 [[ 50]
  [ 55]
  [ 60]]

 [[ 65]
  [ 70]
  [ 75]]

 [[ 80]
  [ 85]
  [ 90]]

 [[ 95]
  [100]
  [105]]

 [[110]
  [115]
  [120]]

 [[125]
  [130]
  [135]]

 [[140]
  [145]
  [150]]

 [[155]
  [160]
  [165]]

 [[170]
  [175]
  [180]]

 [[185]
  [190]
  [195]]

 [[200]
  [205]
  [210]]

 [[215]
  [220]
  [225]]

 [[230]
  [235]
  [240]]

 [[245]
  [250]
  [255]]

 [[260]
  [265]
  [270]]

 [[275]
  [280]
  [285]]

 [[290]
  [2

## Model

In [14]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    encoder = layers.LSTM(100, activation="relu", return_sequences=False)(inputs)
    repeat = layers.RepeatVector(3)(encoder)
    decoder = layers.LSTM(100, activation='relu', return_sequences=True)(repeat)
    outputs = layers.TimeDistributed(layers.Dense(1))(decoder)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 repeat_vector (RepeatVecto  (None, 3, 100)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 3, 100)            80400     
                                                                 
 time_distributed (TimeDist  (None, 3, 1)              101       
 ributed)                                                        
                                                                 
Total params: 121301 (473.83 KB)
Trainable params: 121301 (47

## Train

In [15]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

Epoch 1/1000
6/6 [==============================] - 5s 83ms/step - loss: 23915.5938 - val_loss: 74663.3125
Epoch 2/1000
6/6 [==============================] - 0s 17ms/step - loss: 20490.2598 - val_loss: 65049.6406
Epoch 3/1000
6/6 [==============================] - 0s 18ms/step - loss: 15093.0586 - val_loss: 35468.7773
Epoch 4/1000
6/6 [==============================] - 0s 18ms/step - loss: 7665.8853 - val_loss: 28204.9414
Epoch 5/1000
6/6 [==============================] - 0s 23ms/step - loss: 6899.1484 - val_loss: 17073.4512
Epoch 6/1000
6/6 [==============================] - 0s 19ms/step - loss: 4631.0952 - val_loss: 14968.4072
Epoch 7/1000
6/6 [==============================] - 0s 19ms/step - loss: 4013.9065 - val_loss: 11664.3027
Epoch 8/1000
6/6 [==============================] - 0s 20ms/step - loss: 3048.8870 - val_loss: 9406.3105
Epoch 9/1000
6/6 [==============================] - 0s 18ms/step - loss: 2258.0027 - val_loss: 5798.8892
Epoch 10/1000
6/6 [==========================

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,5998
batch/learning_rate,0.001
batch/loss,0.38868
epoch/epoch,999


## Prediction

In [16]:
test_input = np.array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[[317.16168]
  [323.8422 ]
  [330.59232]]]
